In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import healpy as hp
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import pickle, json
import astropy.constants as c
import astropy.units as u
from mpmath import *
from sympy import *
import sympy
from sympy.solvers import solve

In [3]:
# symbolic calculation
x = symbols('x')
a = Integral(cos(x)*exp(x), x)
Eq(a, a.doit())

Eq(Integral(exp(x)*cos(x), x), exp(x)*sin(x)/2 + exp(x)*cos(x)/2)

In [4]:
# F statistic jacobian
h0, cosi, psi, phi0 = symbols('h0, cosi, psi, phi0')

# A_+, A_x
Ap = (1/2)*h0*(1+cosi**2)
Ac = h0*cosi

# amplitude parameters (Prix & Krishnan)
A1 = +Ap*cos(2*psi)*cos(phi0) - Ac*sin(2*psi)*sin(phi0)
A2 = +Ap*sin(2*psi)*cos(phi0) + Ac*cos(2*psi)*sin(phi0)
A3 = -Ap*cos(2*psi)*sin(phi0) - Ac*sin(2*psi)*cos(phi0)
A4 = -Ap*sin(2*psi)*sin(phi0) + Ac*cos(2*psi)*cos(phi0)
display(A1, A2, A3, A4)

# calculate jacobian matrix
A = Matrix([A1, A2, A3, A4])
Abar = Matrix([h0, cosi, psi, phi0])
Jmat = A.jacobian(Abar)
display(Jmat)

# take absolute value of determinant of Jacobian matrix
jac = Jmat.det()
jac = abs(jac)
jac = jac.simplify()
display(jac)

-cosi*h0*sin(phi0)*sin(2*psi) + 0.5*h0*(cosi**2 + 1)*cos(phi0)*cos(2*psi)

cosi*h0*sin(phi0)*cos(2*psi) + 0.5*h0*(cosi**2 + 1)*sin(2*psi)*cos(phi0)

-cosi*h0*sin(2*psi)*cos(phi0) - 0.5*h0*(cosi**2 + 1)*sin(phi0)*cos(2*psi)

cosi*h0*cos(phi0)*cos(2*psi) - 0.5*h0*(cosi**2 + 1)*sin(phi0)*sin(2*psi)

Matrix([
[ -cosi*sin(phi0)*sin(2*psi) + (0.5*cosi**2 + 0.5)*cos(phi0)*cos(2*psi),  1.0*cosi*h0*cos(phi0)*cos(2*psi) - h0*sin(phi0)*sin(2*psi), -2*cosi*h0*sin(phi0)*cos(2*psi) - 1.0*h0*(cosi**2 + 1)*sin(2*psi)*cos(phi0), -cosi*h0*sin(2*psi)*cos(phi0) - 0.5*h0*(cosi**2 + 1)*sin(phi0)*cos(2*psi)],
[  cosi*sin(phi0)*cos(2*psi) + (0.5*cosi**2 + 0.5)*sin(2*psi)*cos(phi0),  1.0*cosi*h0*sin(2*psi)*cos(phi0) + h0*sin(phi0)*cos(2*psi), -2*cosi*h0*sin(phi0)*sin(2*psi) + 1.0*h0*(cosi**2 + 1)*cos(phi0)*cos(2*psi),  cosi*h0*cos(phi0)*cos(2*psi) - 0.5*h0*(cosi**2 + 1)*sin(phi0)*sin(2*psi)],
[-cosi*sin(2*psi)*cos(phi0) + (-0.5*cosi**2 - 0.5)*sin(phi0)*cos(2*psi), -1.0*cosi*h0*sin(phi0)*cos(2*psi) - h0*sin(2*psi)*cos(phi0), -2*cosi*h0*cos(phi0)*cos(2*psi) + 1.0*h0*(cosi**2 + 1)*sin(phi0)*sin(2*psi),  cosi*h0*sin(phi0)*sin(2*psi) - 0.5*h0*(cosi**2 + 1)*cos(phi0)*cos(2*psi)],
[ cosi*cos(phi0)*cos(2*psi) + (-0.5*cosi**2 - 0.5)*sin(phi0)*sin(2*psi), -1.0*cosi*h0*sin(phi0)*sin(2*psi) + h0*cos(phi0)*cos(2*ps

0.25*Abs(h0**3*(cosi - 1)**3*(cosi + 1)**3)

In [5]:
# determining constants for physical and canonical prior
h0, cosi, psi, phi0, h0max = symbols('h0, cosi, psi, phi0, h0max')

int1 = integrate(1, (h0, 0, h0max))
int2 = integrate(1, (cosi, -1, 1))
int3 = integrate(1, (psi, -pi/4, pi/4))
int4 = integrate(1, (phi0, 0, 2*pi))
integral = int1*int2*int3*int4

C_physical = 1/integral
display(C_physical)

# for canonical prior, use jacobian (1/4)h_0^3(1-cos^2i)^3
int1 = integrate((1/4)*h0**3, (h0, 0, h0max))
int2 = integrate((1-cosi**2)**3, (cosi, -1, 1))
int3 = integrate(1, (psi, -pi/4, pi/4))
int4 = integrate(1, (phi0, 0, 2*pi))
integral = int1*int2*int3*int4

C_canonical = 1/integral
display(C_canonical)

1/(2*pi**2*h0max)

17.5/(pi**2*h0max**4)

In [6]:
# expected value of squared SNR
a, b, c = symbols('a, b, c')

rho2_orig = a*(A1**2 + A3**2) + b*(A2**2 + A4**2) + 2*c*(A1*A2 + A3*A4)

# simplified form
x = a*(Ap**2 *(cos(2*psi))**2 + Ac**2 * (sin(2*psi))**2)
y = b*(Ap**2 *(sin(2*psi))**2 + Ac**2 * (cos(2*psi))**2) 
z = 2*c*(Ap**2-Ac**2)*cos(2*psi)*sin(2*psi)
rho2 = x+y+z
display(rho2)

# check that they give the same expressions
delta = rho2-rho2_orig
delta = delta.simplify()
display(delta)

# integrate over cosi, psi, phi0
int1 = (2/pi)*integrate(rho2, (psi, -pi/4, pi/4))
int1 = int1.simplify()
int2 = (1/2)*integrate(int1, (cosi, -1, 1))
int2 = int2.simplify()
display(int2)

a*(cosi**2*h0**2*sin(2*psi)**2 + 0.25*h0**2*(cosi**2 + 1)**2*cos(2*psi)**2) + b*(cosi**2*h0**2*cos(2*psi)**2 + 0.25*h0**2*(cosi**2 + 1)**2*sin(2*psi)**2) + 2*c*(-cosi**2*h0**2 + 0.25*h0**2*(cosi**2 + 1)**2)*sin(2*psi)*cos(2*psi)

0

0.4*h0**2*(a + b)

In [7]:
# integrate L(A(Abar)) d^4 Abar
a, b, c, h0max = symbols('a, b, c, h0max')
x1, x2, x3, x4 = symbols('x1, x2, x3, x4')

# inner product (x|s)
xs = x1*A1 + x2*A2 + x3*A3 +x4*A4
display(xs)

# inner product (s|s)
x = a*(Ap**2 *(cos(2*psi))**2 + Ac**2 * (sin(2*psi))**2)
y = b*(Ap**2 *(sin(2*psi))**2 + Ac**2 * (cos(2*psi))**2) 
z = 2*c*(Ap**2-Ac**2)*cos(2*psi)*sin(2*psi)
ss = x+y+z
display(ss)

# integrand
f = exp(xs - (1/2)*ss)
display(f)

# integrate
#int1 = (1/(2*pi))*integrate(f, (phi0, 0, 2*pi))
#int2 = (1/2)*integrate(int1, (cosi, -1, 1))
#int3 = (2/pi)*integrate(int2, (psi, -pi/4, pi/4))
#int4 = (1/h0max)*integrate(int3, (h0, 0, h0max))

x1*(-cosi*h0*sin(phi0)*sin(2*psi) + 0.5*h0*(cosi**2 + 1)*cos(phi0)*cos(2*psi)) + x2*(cosi*h0*sin(phi0)*cos(2*psi) + 0.5*h0*(cosi**2 + 1)*sin(2*psi)*cos(phi0)) + x3*(-cosi*h0*sin(2*psi)*cos(phi0) - 0.5*h0*(cosi**2 + 1)*sin(phi0)*cos(2*psi)) + x4*(cosi*h0*cos(phi0)*cos(2*psi) - 0.5*h0*(cosi**2 + 1)*sin(phi0)*sin(2*psi))

a*(cosi**2*h0**2*sin(2*psi)**2 + 0.25*h0**2*(cosi**2 + 1)**2*cos(2*psi)**2) + b*(cosi**2*h0**2*cos(2*psi)**2 + 0.25*h0**2*(cosi**2 + 1)**2*sin(2*psi)**2) + 2*c*(-cosi**2*h0**2 + 0.25*h0**2*(cosi**2 + 1)**2)*sin(2*psi)*cos(2*psi)

exp(-0.5*a*(cosi**2*h0**2*sin(2*psi)**2 + 0.25*h0**2*(cosi**2 + 1)**2*cos(2*psi)**2) - 0.5*b*(cosi**2*h0**2*cos(2*psi)**2 + 0.25*h0**2*(cosi**2 + 1)**2*sin(2*psi)**2) - 1.0*c*(-cosi**2*h0**2 + 0.25*h0**2*(cosi**2 + 1)**2)*sin(2*psi)*cos(2*psi) + x1*(-cosi*h0*sin(phi0)*sin(2*psi) + 0.5*h0*(cosi**2 + 1)*cos(phi0)*cos(2*psi)) + x2*(cosi*h0*sin(phi0)*cos(2*psi) + 0.5*h0*(cosi**2 + 1)*sin(2*psi)*cos(phi0)) + x3*(-cosi*h0*sin(2*psi)*cos(phi0) - 0.5*h0*(cosi**2 + 1)*sin(phi0)*cos(2*psi)) + x4*(cosi*h0*cos(phi0)*cos(2*psi) - 0.5*h0*(cosi**2 + 1)*sin(phi0)*sin(2*psi)))

In [8]:
# express jacobian in terms of A1, A2, A3, A4
A1, A2, A3, A4 = symbols('A1, A2, A3, A4')

# inverse transformation formulas for h0, cosi
Ap = (1/2)*(sqrt((A1+A4)**2 + (A2-A3)**2) + sqrt((A1-A4)**2 + (A2+A3)**2))
Ac = (1/2)*(sqrt((A1+A4)**2 + (A2-A3)**2) - sqrt((A1-A4)**2 + (A2+A3)**2))

cosi = Ap/Ac - sqrt((Ap/Ac)**2 - 1)
h0 = Ac/cosi

display(Ap, Ac, cosi, h0)

0.5*sqrt((A1 - A4)**2 + (A2 + A3)**2) + 0.5*sqrt((A1 + A4)**2 + (A2 - A3)**2)

-0.5*sqrt((A1 - A4)**2 + (A2 + A3)**2) + 0.5*sqrt((A1 + A4)**2 + (A2 - A3)**2)

-sqrt(-1 + 1.0*(sqrt((A1 - A4)**2 + (A2 + A3)**2) + sqrt((A1 + A4)**2 + (A2 - A3)**2))**2/(-sqrt((A1 - A4)**2 + (A2 + A3)**2) + sqrt((A1 + A4)**2 + (A2 - A3)**2))**2) + (0.5*sqrt((A1 - A4)**2 + (A2 + A3)**2) + 0.5*sqrt((A1 + A4)**2 + (A2 - A3)**2))/(-0.5*sqrt((A1 - A4)**2 + (A2 + A3)**2) + 0.5*sqrt((A1 + A4)**2 + (A2 - A3)**2))

(-0.5*sqrt((A1 - A4)**2 + (A2 + A3)**2) + 0.5*sqrt((A1 + A4)**2 + (A2 - A3)**2))/(-sqrt(-1 + 1.0*(sqrt((A1 - A4)**2 + (A2 + A3)**2) + sqrt((A1 + A4)**2 + (A2 - A3)**2))**2/(-sqrt((A1 - A4)**2 + (A2 + A3)**2) + sqrt((A1 + A4)**2 + (A2 - A3)**2))**2) + (0.5*sqrt((A1 - A4)**2 + (A2 + A3)**2) + 0.5*sqrt((A1 + A4)**2 + (A2 - A3)**2))/(-0.5*sqrt((A1 - A4)**2 + (A2 + A3)**2) + 0.5*sqrt((A1 + A4)**2 + (A2 - A3)**2)))